# Spark Job Notebook
This template initializes PySpark so you can validate Spark jobs before sending them from the Forestrat UI.## Quick Start
- Run the setup cell below to start a local PySpark session with a sample dataset.
- Modify or add cells to prototype your Spark transformations.
- Tag the cell you want to send with `api-send` (already applied to the last cell).
- Use the **Send Code to API** button at the bottom when you are ready to dispatch the job.> The kernel and PySpark session start automatically when you run the first code cell.

In [ ]:
# Initialize PySpark Execution Engine
from forestrat_lib import (
    describe_dataframe,
    detect_outliers_iqr,
    ensure_datetime,
    normalize_column,
    profile_dataframe,
    top_categorical_values,
    zscore,
)

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count, when

spark = SparkSession.builder \
    .appName("ForestratSparkNotebook") \
    .master("local[*]") \
    .getOrCreate()

print("✅ PySpark initialized successfully!")
print(f"📊 Spark Version: {spark.version}")
print(f"🔧 Spark Context: {spark.sparkContext}")
print(f"🌐 Web UI: {spark.sparkContext.uiWebUrl}")

sample_df = spark.createDataFrame(
    [
        (1, "Alice", 25, "Engineer"),
        (2, "Bob", 30, "Manager"),
        (3, "Charlie", 35, "Analyst"),
        (4, "Diana", 28, "Designer"),
        (5, "Eve", 32, "Developer"),
    ],
    ["id", "name", "age", "role"],
)

print("\n📊 Sample Dataset:")
sample_df.show()

enriched_df = sample_df.withColumn(
    "age_group",
    when(col("age") < 30, "Young").otherwise("Experienced")
)

print("\n✨ Enriched Dataset:")
enriched_df.show()

summary_df = enriched_df.groupBy("age_group").agg(
    count("*").alias("count"),
    avg("age").alias("avg_age")
)

print("\n📈 Summary Metrics:")
summary_df.show()


In [ ]:
import ipywidgets as widgets
from IPython.display import display
import requests
import json
import time
from datetime import datetime
import os

output = widgets.Output()

def send_code_to_api(button):
    with output:
        output.clear_output(wait=True)
        
        try:
            print("🔍 Searching for cell with tag 'api-send'...")
            
            # Read the notebook file directly
            import nbformat
            from nbformat import NotebookNode
            
            # Get current notebook path
            notebook_files = [f for f in os.listdir('.') if f.endswith('.ipynb')]
            
            if not notebook_files:
                print("❌ Error: Could not find notebook file")
                return
            
            # Try to find the current notebook
            notebook_path = None
            for nb_file in notebook_files:
                try:
                    with open(nb_file, 'r', encoding='utf-8') as f:
                        nb = nbformat.read(f, as_version=4)
                        # Check if this notebook has the api-send tag
                        for i, cell in enumerate(nb.cells):
                            tags = cell.metadata.get('tags', [])
                            if 'api-send' in tags:
                                notebook_path = nb_file
                                break
                        if notebook_path:
                            break
                except:
                    continue
            
            if not notebook_path:
                print("❌ Error: Could not find notebook with cell tagged 'api-send'")
                return
            
            # Read the notebook
            with open(notebook_path, 'r', encoding='utf-8') as f:
                nb = nbformat.read(f, as_version=4)
            
            # Find cell with api-send tag
            code = None
            cell_index = -1
            cell_type = None
            
            for i, cell in enumerate(nb.cells):
                tags = cell.metadata.get('tags', [])
                if 'api-send' in tags:
                    cell_index = i
                    cell_type = cell.cell_type
                    source = cell.source
                    if isinstance(source, list):
                        code = ''.join(source)
                    else:
                        code = source
                    break
            
            if code is None:
                print("❌ Error: No cell found with tag 'api-send'")
                return
            
            if not code.strip():
                print("⚠️ Warning: Cell with tag 'api-send' is empty")
                return
            
            print(f"✅ Found cell #{cell_index} (type: {cell_type}) with code ({len(code)} characters)")
            
            # Prepare API request
            api_url = 'http://18.235.156.98:12345/contentListener'
            
            print(f"\n🚀 Sending code to API...")
            print(f"📍 URL: {api_url}")
            
            # Get job name and cron schedule from input widgets
            job_name = job_name_input.value.strip() if job_name_input.value else None
            cron_schedule = cron_schedule_input.value.strip() if cron_schedule_input.value else None
            
            payload = {
                'code': code,
                'metadata': {
                    'language': 'python',
                    'source': 'jupyter_notebook',
                    'cell_index': cell_index,
                    'cell_type': cell_type,
                    'timestamp': datetime.now().isoformat(),
                    'code_length': len(code)
                }
            }
            
            if job_name:
                payload['job_name'] = job_name
            if cron_schedule:
                payload['cron_schedule'] = cron_schedule
            
            headers = {'Content-Type': 'application/json'}
            
            try:
                response = requests.post(api_url, json=payload, headers=headers, timeout=30)
                response.raise_for_status()
                result = response.json()
                print(f"\n✅ Success! API Response:")
                print(json.dumps(result, indent=2, ensure_ascii=False))
            except Exception as e:
                print(f"\n❌ Error: {str(e)}")
                
        except Exception as e:
            print(f"\n❌ Critical Error: {str(e)}")
            import traceback
            traceback.print_exc()

# Create input widgets
job_name_input = widgets.Text(value='', placeholder='Enter job name', description='Job Name:', style={'description_width': 'initial'}, layout=widgets.Layout(width='500px'))
cron_schedule_input = widgets.Text(value='', placeholder='Enter cron schedule', description='Cron Schedule:', style={'description_width': 'initial'}, layout=widgets.Layout(width='500px'))
button = widgets.Button(description="📤 Send Code to API", button_style="success", icon="send", layout=widgets.Layout(width="250px", height="45px"))
button.on_click(send_code_to_api)

# Display widgets
display(widgets.VBox([
    widgets.HTML("<h3>📤 Send Code to API</h3>"),
    job_name_input,
    cron_schedule_input,
    button,
    output
]))

print("📋 Instructions:")
print("─" * 50)
print("1. Tag a cell with 'api-send'")
print("2. Enter job name (optional)")
print("3. Enter cron schedule (optional)")
print("4. Click the button above to send the code")
print("─" * 50)
